In [1]:
# Packages

# To make data frames
import pandas as pd

# Timer
import time # for debugging

# For date arithmetic
from datetime import datetime, timedelta

In [2]:
# Functions

# Get how long a function takes (needs time)
def time_elapsed(start, end):
    total_time_seconds = end-start
    print("Total time elapsed:") 
    if total_time_seconds < 60:
        print(total_time_seconds, "seconds")
    else:
        total_time_minutes = total_time_seconds/60 
        if total_time_minutes < 60: 
            print(total_time_minutes, "minutes") 
        else: 
            total_time_hours = total_time_minutes/60 
            print(total_time_hours, "hours")
    print() 
    
# Add a row to df with given end_date, candidate_name, and party (needs df)
def insert_row(date, candidate, pop):
    global df
    df.at[len(df), 'end_date'] = date
    df.at[len(df)-1, 'candidate_name'] = candidate
    df.at[len(df)-1, 'population'] = pop

# Impute a missing date for a candidate (needs the df)
def impute_for_candidate(date, candidate, pop):
    print("date, candidate, pop are "+str(date)+", "+candidate+", "+pop) # for debugging
    if ((df['end_date'] == date) & (df['candidate_name'] == candidate) & (df['population'] == pop)).any() == False:   
        #print("No entry for this date, candidate, population.") # for debugging
        insert_row(date, candidate, pop)    
        #print("Now we've inserted a row.") # for debugging
    #else: # for debugging
        #print("Candidate has an entry here.") # for debugging
    #print() # for debugging

In [3]:
# Read in the data

#old_data = pd.read_csv("president_polls_cleaned.csv")
#new_data = pd.read_csv("president_polls.csv")
df = pd.read_csv("president_polls.csv")#_cleaned.csv")
#df = pd.merge(old_data, new_data, how='left')

In [4]:
# Check for any redundant columns and for number of unique entries

df.nunique()

poll_id                      2409
pollster_id                   181
pollster                      181
sponsor_ids                   213
sponsors                      213
display_name                  181
pollster_rating_id            172
pollster_rating_name          172
numeric_grade                  24
pollscore                      23
methodology                    46
transparency_score             19
state                          53
start_date                    806
end_date                      806
sponsor_candidate_id           12
sponsor_candidate              12
sponsor_candidate_party         4
endorsed_candidate_id           0
endorsed_candidate_name         0
endorsed_candidate_party        0
question_id                  4619
sample_size                  1038
population                      4
subpopulation                   0
population_full                 4
tracking                        1
created_at                   1581
notes                          20
url           

In [5]:
# Delete superfluous columns

df.drop(columns=['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'display_name', 'pollster_rating_id', 
                 'sponsor_candidate_id', 'sponsor_candidate', 'sponsor_candidate_party', 'endorsed_candidate_id', 
                 'endorsed_candidate_name', 'endorsed_candidate_party', 'subpopulation', 'population_full', 'tracking', 
                 'created_at', 'notes', 'url', 'source', 'internal', 'race_id', 'cycle', 'office_type', 'seat_number', 
                 'seat_name', 'election_date', 'stage', 'nationwide_batch', 'ranked_choice_reallocated', 
                 'ranked_choice_round', 'answer', 'candidate_id'], inplace=True)

In [6]:
# Parse the dates in the end_date column

df['end_date'] = pd.to_datetime(df['end_date'])

In [7]:
# Replace the 'v' in the population column with 'rv'

df['population'].replace('v', 'rv', inplace=True)

In [8]:
# Use groupby to get daily averages according to population polled

cand_avg_by_date = df.groupby(['end_date', 'candidate_name', 'population'])['pct'].mean().reset_index(name='da_by_pop')
df = pd.merge(df, cand_avg_by_date)

In [9]:
# Get lists for imputing 

all_dates = pd.date_range(df['end_date'].tolist()[-1], df['end_date'].tolist()[0], freq='D')
candidates = ['Donald Trump', 'Joe Biden', 'Kamala Harris'] 
pops = ['lv', 'rv', 'a']

In [10]:
# Impute the missing dates, candidate, populations 

start_impute = time.time()
for date in all_dates:
    for candidate in candidates:
        for pop in pops:
            impute_for_candidate(date, candidate, pop)
end_impute = time.time()
time_elapsed(start_impute, end_impute)   

date, candidate, pop are 2021-04-07 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-04-07 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-04-07 00:00:00, Donald Trump, a
date, candidate, pop are 2021-04-07 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-04-07 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-04-07 00:00:00, Joe Biden, a
date, candidate, pop are 2021-04-07 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-04-07 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-04-07 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-04-08 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-04-08 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-04-08 00:00:00, Donald Trump, a
date, candidate, pop are 2021-04-08 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-04-08 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-04-08 00:00:00, Joe Biden, a
date, candidate, pop are 2021-04-08 00:00:00, Kamala Harris, lv
date, candi

date, candidate, pop are 2021-04-27 00:00:00, Joe Biden, a
date, candidate, pop are 2021-04-27 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-04-27 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-04-27 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-04-28 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-04-28 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-04-28 00:00:00, Donald Trump, a
date, candidate, pop are 2021-04-28 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-04-28 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-04-28 00:00:00, Joe Biden, a
date, candidate, pop are 2021-04-28 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-04-28 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-04-28 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-04-29 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-04-29 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-04-29 00:00:00, Donald Trump, a
date

date, candidate, pop are 2021-05-16 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-05-16 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-05-16 00:00:00, Joe Biden, a
date, candidate, pop are 2021-05-16 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-05-16 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-05-16 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-05-17 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-05-17 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-05-17 00:00:00, Donald Trump, a
date, candidate, pop are 2021-05-17 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-05-17 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-05-17 00:00:00, Joe Biden, a
date, candidate, pop are 2021-05-17 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-05-17 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-05-17 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-05-18 00:00:00, Donald Trump, lv
date, can

date, candidate, pop are 2021-06-04 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-06-04 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-06-05 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-06-05 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-06-05 00:00:00, Donald Trump, a
date, candidate, pop are 2021-06-05 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-06-05 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-06-05 00:00:00, Joe Biden, a
date, candidate, pop are 2021-06-05 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-06-05 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-06-05 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-06-06 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-06-06 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-06-06 00:00:00, Donald Trump, a
date, candidate, pop are 2021-06-06 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-06-06 00:00:00, Joe Biden, rv
date, c

date, candidate, pop are 2021-06-21 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-06-22 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-06-22 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-06-22 00:00:00, Donald Trump, a
date, candidate, pop are 2021-06-22 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-06-22 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-06-22 00:00:00, Joe Biden, a
date, candidate, pop are 2021-06-22 00:00:00, Kamala Harris, lv
date, candidate, pop are 2021-06-22 00:00:00, Kamala Harris, rv
date, candidate, pop are 2021-06-22 00:00:00, Kamala Harris, a
date, candidate, pop are 2021-06-23 00:00:00, Donald Trump, lv
date, candidate, pop are 2021-06-23 00:00:00, Donald Trump, rv
date, candidate, pop are 2021-06-23 00:00:00, Donald Trump, a
date, candidate, pop are 2021-06-23 00:00:00, Joe Biden, lv
date, candidate, pop are 2021-06-23 00:00:00, Joe Biden, rv
date, candidate, pop are 2021-06-23 00:00:00, Joe Biden, a
date, candid

...

date, candidate, pop are 2024-05-25 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-05-25 00:00:00, Joe Biden, a
date, candidate, pop are 2024-05-25 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-05-25 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-05-25 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-05-26 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-05-26 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-05-26 00:00:00, Donald Trump, a
date, candidate, pop are 2024-05-26 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-05-26 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-05-26 00:00:00, Joe Biden, a
date, candidate, pop are 2024-05-26 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-05-26 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-05-26 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-05-27 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-05-27 00:00:00, Donald Trump, rv
date, 

date, candidate, pop are 2024-06-12 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-06-12 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-06-12 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-06-13 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-06-13 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-06-13 00:00:00, Donald Trump, a
date, candidate, pop are 2024-06-13 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-06-13 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-06-13 00:00:00, Joe Biden, a
date, candidate, pop are 2024-06-13 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-06-13 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-06-13 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-06-14 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-06-14 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-06-14 00:00:00, Donald Trump, a
date, candidate, pop are 2024-06-14 00:00:00, Joe Biden, lv
dat

date, candidate, pop are 2024-06-29 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-06-29 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-06-29 00:00:00, Donald Trump, a
date, candidate, pop are 2024-06-29 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-06-29 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-06-29 00:00:00, Joe Biden, a
date, candidate, pop are 2024-06-29 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-06-29 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-06-29 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-06-30 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-06-30 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-06-30 00:00:00, Donald Trump, a
date, candidate, pop are 2024-06-30 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-06-30 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-06-30 00:00:00, Joe Biden, a
date, candidate, pop are 2024-06-30 00:00:00, Kamala Harris, lv
date, candi

date, candidate, pop are 2024-07-16 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-07-16 00:00:00, Donald Trump, a
date, candidate, pop are 2024-07-16 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-07-16 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-07-16 00:00:00, Joe Biden, a
date, candidate, pop are 2024-07-16 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-07-16 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-07-16 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-07-17 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-07-17 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-07-17 00:00:00, Donald Trump, a
date, candidate, pop are 2024-07-17 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-07-17 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-07-17 00:00:00, Joe Biden, a
date, candidate, pop are 2024-07-17 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-07-17 00:00:00, Kamala Harris, rv
date, cand

date, candidate, pop are 2024-07-31 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-07-31 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-07-31 00:00:00, Donald Trump, a
date, candidate, pop are 2024-07-31 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-07-31 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-07-31 00:00:00, Joe Biden, a
date, candidate, pop are 2024-07-31 00:00:00, Kamala Harris, lv
date, candidate, pop are 2024-07-31 00:00:00, Kamala Harris, rv
date, candidate, pop are 2024-07-31 00:00:00, Kamala Harris, a
date, candidate, pop are 2024-08-01 00:00:00, Donald Trump, lv
date, candidate, pop are 2024-08-01 00:00:00, Donald Trump, rv
date, candidate, pop are 2024-08-01 00:00:00, Donald Trump, a
date, candidate, pop are 2024-08-01 00:00:00, Joe Biden, lv
date, candidate, pop are 2024-08-01 00:00:00, Joe Biden, rv
date, candidate, pop are 2024-08-01 00:00:00, Joe Biden, a
date, candidate, pop are 2024-08-01 00:00:00, Kamala Harris, lv
date, candi

In [11]:
# Fill in the missing averages

df.sort_values(['candidate_name', 'population', 'end_date'], inplace=True)
df['da_by_pop'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.bfill())

In [12]:
df.loc[(df['candidate_name'] == 'Donald Trump') | (df['candidate_name'] == 'Joe Biden') | 
       (df['candidate_name'] == 'Kamala Harris'), ['population', 'candidate_name', 
                                                   'end_date', 'da_by_pop']].sample(5)

,population,candidate_name,end_date,da_by_pop
3030,rv,Joe Biden,2024-05-21,40.311111
14738,lv,Joe Biden,2022-03-28,38.000000
17194,a,Kamala Harris,2023-03-09,37.750000
3076,lv,Joe Biden,2024-05-20,55.166667
8134,rv,Donald Trump,2023-10-02,44.666667


In [13]:
# Add columns for rolling averages

df.sort_values(['candidate_name', 'population', 'end_date'], inplace=True)
df['pop_raverage3'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.rolling(3, 1).mean())
df['pop_raverage4'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.rolling(4, 1).mean())
df['pop_raverage5'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.rolling(5, 1).mean())
df['pop_raverage6'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.rolling(6, 1).mean())
df['pop_raverage7'] = df.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.rolling(7, 1).mean())                       

In [14]:
# Format populations

df['population'].replace(['a', 'rv', 'lv'], ['All adults', ' Registered voters', 'Likely voters'], inplace=True)

In [15]:
# Sort again

df.sort_values(by=['end_date', 'candidate_name', 'population'], ascending=[False, True, True], inplace=True)
#df = df.reset_index(drop=True)

In [16]:
# Export cleaned data for use in Tableau

df.to_csv("president_polls_cleaned.csv", index=False)